# Install Libraries

In [1]:
!pip install selenium

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

# Work

In [2]:
binary = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options = Options()
options.headless = True
options.binary = binary

cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True #optional

driver = webdriver.Firefox(options=options, capabilities=cap)
#driver = webdriver.Firefox(capabilities=cap)
driver.get("http://diariooficial.rn.gov.br/dei/dorn3/")

In [10]:
materias = []

In [11]:
def navegar(link, blank=5):
    link.click()
    global materias
    subsessoes = []
    try:
        subsessoes = driver.find_element_by_id("lblQtdMateriasPublicadas")
        if subsessoes.text == 'Nenhuma matéria encontrada nessa data para essa seção.':
            caminho = driver.find_element_by_id("lblSessaoPath").text
            lista_caminho = caminho.split("\\")[:-1]
            if blank != 0:
                [lista_caminho.append('') for b in range(blank)]
            if lista_caminho[0] == "DIÁRIO DA JUSTIÇA":
                lista_caminho.insert(0,'DIÁRIO OFICIAL')
            lista_caminho.append(subsessoes.text)
            print(lista_caminho)
            materias.append(lista_caminho)
            #print(materias)
            return []
        tabela = driver.find_element_by_id("dgDocumentos")
        files = tabela.find_elements_by_tag_name("a")
        for f in files:
            link = f.get_attribute("href")
            caminho = driver.find_element_by_id("lblSessaoPath").text
            lista_caminho = caminho.split("\\")[:-1]
            if blank != 0:
                [lista_caminho.append('') for b in range(blank)]
            if lista_caminho[0] == "DIÁRIO DA JUSTIÇA":
                lista_caminho.insert(0,'DIÁRIO OFICIAL')
            lista_caminho.append(link)
            print(lista_caminho)
            materias.append(lista_caminho)
        return []
    except:
        tabela = driver.find_element_by_id("dgSubSessoes")
        links = tabela.find_elements_by_tag_name("a")
    return links

#elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
#teste = navegar(elements)
#materias

In [12]:
def crawl_diario(link_sessao):
    #sleep(1)
    sub1 = navegar(link_sessao)
    if len(sub1) != 0:
        id1 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub1)+2)]
        for i1 in id1:
            #sleep(1)
            #print(i1)
            s1 = driver.find_element_by_id(i1)
            sub2 = navegar(s1, blank=4)
            if len(sub2) == 0:
                #sleep(1)
                driver.back()
            else:
                id2 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                       else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub2)+2)]
                for i2 in id2:
                    #sleep(1)
                    s2 = driver.find_element_by_id(i2)
                    sub3 = navegar(s2, blank=3)
                    if len(sub3) == 0:
                        #sleep(1)
                        driver.back()
                    else:
                        id3 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub3)+2)]
                        for i3 in id3:
                            #sleep(1)
                            s3 = driver.find_element_by_id(i3)
                            sub4 = navegar(s3, blank=2)
                            if len(sub4) == 0:
                                #sleep(1)
                                driver.back()
                            else:
                                id4 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                                       else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub4)+2)]
                                for i4 in id4:
                                    #sleep(1)
                                    s4 = driver.find_element_by_id(i4)
                                    sub5 = navegar(s4, blank=1)
                                    if len(sub5) == 0:
                                        #sleep(1)
                                        driver.back()
                                    else:
                                        id5 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                                               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub5)+2)]
                                        for i5 in id5:
                                            #sleep(1)
                                            s5 = driver.find_element_by_id(i5)
                                            sub6 = navegar(s5, blank=0)
                                            if len(sub6) == 0:
                                                #sleep(1)
                                                driver.back()
                                            if i5 == id5[-1]:
                                                #sleep(1)
                                                driver.back()
                                    if i4 == id4[-1]:
                                        #sleep(1)
                                        driver.back()
                            if i3 == id3[-1]:
                                #sleep(1)
                                driver.back()
                    if i2 == id2[-1]:
                        #sleep(1)
                        driver.back()
            if i1 == id1[-1]:
                #sleep(1)
                driver.back()

segunda versão

In [13]:
elements = ["Menu1_dgMenu_ctl0" + str(i) + "_LinkButton1" for i in range(2,10)]
#lista = navegar(sessoes[7])
#lista = []
for e in elements:
    s = driver.find_element_by_id(e)
    crawl_diario(s)
print(materias)
#lista

['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (ASSECOM)', 'DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI)', 'DEI ASSEJUR', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE)', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONSULTORIA GERAL DO ESTADO (CGE)', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONTROLADORIA GERAL DO ESTADO (CONTROL)', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'DEFENSORIA PÚBLICA DO ESTADO (DPE)', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIV

['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TRIBUTAÇÃO', 'SECRETARIA DE ESTADO DE TRIBUTACAO-SUMATI', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TRIBUTAÇÃO', 'SECRETARIA DE TRIBUTAÇÃO / 1 URT', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TRIBUTAÇÃO', 'SECRETARIA DE TRIBUTAÇÃO / 2 URT', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TRIBUTAÇÃO', 'SECRETARIA DE TRIBUTAÇÃO / 3 URT', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TRIBUTAÇÃO', 'SECRETARIA DE TRIBUTAÇÃO / 4 URT', '', '', 'Nenhuma matéria encontrada nessa data para essa seção

['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE)', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'CONSULTORIA GERAL DO ESTADO (CGE)', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'CONTROLADORIA GERAL DO ESTADO (CONTROL)', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'DECRETOS', '', '', '', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660281']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'DECRETOS', '', '', '', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660282']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'GOVERNADORIA', 'DECRETOS', '', '', '', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=000

['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DE ESTADO DA SAÚDE PÚBLICA (SESAP)', '', '', '', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660238']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DE ESTADO DA SAÚDE PÚBLICA (SESAP)', '', '', '', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660248']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETÁRIA DE ESTADO DAS MULHERES, DA JUVENTUDE, DA IGUALDADE RACIAL E DOS DIREITOS HUMANOS (SEMJIDH)', 'SECRETÁRIA DE ESTADO DAS MULHERES, DA JUVENTUDE, DA IGUALDADE RACIAL E DOS DIREITOS HUMANOS (SEMJIDH)', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETÁRIA DE ESTADO DO DESENVOLVIMENTO RURAL E DA AGRICULTURA FAMILIAR (SEDRAF)', 'SECRETÁRIA DE ESTADO DO DESENVOLVIMENTO RURAL E DA AGRICULTURA 

['DIÁRIO OFICIAL', 'PODER LEGISLATIVO', 'DIÁRIO DA ASSEMBLÉIA LEGISLATIVA ', 'SECRETARIA LEGISLATIVA', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'MINISTÉRIO PÚBLICO', '', '', '', '', '', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660291']
['DIÁRIO OFICIAL', 'MINISTÉRIO PÚBLICO', '', '', '', '', '', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660292']
['DIÁRIO OFICIAL', 'MINISTÉRIO PÚBLICO', '', '', '', '', '', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660293']
['DIÁRIO OFICIAL', 'TRIBUNAL DE CONTAS', '', '', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'PREFEITURAS', '', '', '', '', '', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660265']
['DIÁRIO OFICIAL', 'PREFEITURAS', '', '', '', '', '', 'http://diari

['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS CRIMINAIS ', '1ª VARA CRIMINAL ', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS CRIMINAIS ', '2ª VARA CRIMINAL ', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS CRIMINAIS ', '3ª VARA CRIMINAL ', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS CRIMINAIS ', '4ª VARA CRIMINAL ', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JU

['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '1ª VARA DE FAZENDA PÚBLICA ', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '1ª VARA DE SUCESSÕES ', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '1º VARA DE EXECUÇÃO FISCAL ESTADUAL E TRIBUTÁRIA', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '2ª VARA DA INFÂNCIA E DA JUVENTUDE ', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '2ª VARA DE EXECUCAO FISCAL MUN

['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CRIMINAIS ', '2ª VARA CRIMINAL ', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CRIMINAIS ', '3ª VARA CRIMINAL ', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CRIMINAIS ', '4ª VARA CRIMINAL ', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CRIMINAIS ', '5ª VARA CRIMINAL ', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CRIMINAIS ', '6ª VARA CRIMINAL ', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COM

['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '2ª VARA FAMILIA', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '3ª VARA CÍVEL', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '3ª VARA CRIMINAL', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '3ª VARA FAMÍLIA', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '4ª VARA CÍVEL', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'PUBLICAÇÕES A PEDIDO', 'COMARCA DE MOSSORO', '4ª VARA CRIMINAL', '', '', 'Nenhuma matéria encontrada nessa 

['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TRIBUNAL PLENO E GABINETES', 'AVISOS', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TRIBUNAL PLENO E GABINETES', 'DESPACHOS', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'TRIBUNAL PLENO E GABINETES', 'RESOLUÇÕES ADMINISTRATIVAS', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL ELEITORAL (TRE) ', 'AÇÕES', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL ELEITORAL (TRE) ', 'AGRAVOS', '', '', '', 'Nenhuma matéria encontrada nessa data para essa seção.']
['DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL ELEITORAL (TRE) ', 'APELAÇÕES', '', '', '', 'Ne

In [15]:
df = pd.DataFrame(materias,columns=['diario', 'sessao', 'subsessao1', 'subsessao2', 'subsessao3', 'subsessao4', 'subsessão5', 'materia'])
df.head()

,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,materia
0,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,,Nenhuma matéria encontrada nessa data para ess...
1,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE),,,,Nenhuma matéria encontrada nessa data para ess...
2,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSULTORIA GERAL DO ESTADO (CGE),,,,Nenhuma matéria encontrada nessa data para ess...
3,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONTROLADORIA GERAL DO ESTADO (CONTROL),,,,Nenhuma matéria encontrada nessa data para ess...
4,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",DEFENSORIA PÚBLICA DO ESTADO (DPE),,,,Nenhuma matéria encontrada nessa data para ess...


In [16]:
df[df['sessao'] == 'DIÁRIO DA JUSTIÇA']

,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,materia
186,DIÁRIO OFICIAL,DIÁRIO DA JUSTIÇA,JUSTIÇA ESTADUAL,1ª VARA DE PRECATÓRIAS,,,,Nenhuma matéria encontrada nessa data para ess...
187,DIÁRIO OFICIAL,DIÁRIO DA JUSTIÇA,JUSTIÇA ESTADUAL,COMARCA DA CAPITAL,CENTRAL DE AVALIAÇÃO E ARREMATAÇÃO,,,Nenhuma matéria encontrada nessa data para ess...
188,DIÁRIO OFICIAL,DIÁRIO DA JUSTIÇA,JUSTIÇA ESTADUAL,COMARCA DA CAPITAL,DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VAREL...,JUIZADOS ESPECIAIS CÍVEIS E CRIMINAIS,,Nenhuma matéria encontrada nessa data para ess...
189,DIÁRIO OFICIAL,DIÁRIO DA JUSTIÇA,JUSTIÇA ESTADUAL,COMARCA DA CAPITAL,DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VAREL...,VARAS CRIMINAIS,10ª VARA CRIMINAL,Nenhuma matéria encontrada nessa data para ess...
190,DIÁRIO OFICIAL,DIÁRIO DA JUSTIÇA,JUSTIÇA ESTADUAL,COMARCA DA CAPITAL,DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VAREL...,VARAS CRIMINAIS,11ª VARA CRIMINAL,Nenhuma matéria encontrada nessa data para ess...
191,DIÁRIO OFICIAL,DIÁRIO DA JUSTIÇA,JUSTIÇA ESTADUAL,COMARCA DA CAPITAL,DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VAREL...,VARAS CRIMINAIS,12ª VARA CRIMINAL,Nenhuma matéria encontrada nessa data para ess...
192,DIÁRIO OFICIAL,DIÁRIO DA JUSTIÇA,JUSTIÇA ESTADUAL,COMARCA DA CAPITAL,DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VAREL...,VARAS CRIMINAIS,13ª VARA CRIMINAL,Nenhuma matéria encontrada nessa data para ess...
193,DIÁRIO OFICIAL,DIÁRIO DA JUSTIÇA,JUSTIÇA ESTADUAL,COMARCA DA CAPITAL,DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VAREL...,VARAS CRIMINAIS,14ª VARA CRIMINAL,Nenhuma matéria encontrada nessa data para ess...
194,DIÁRIO OFICIAL,DIÁRIO DA JUSTIÇA,JUSTIÇA ESTADUAL,COMARCA DA CAPITAL,DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VAREL...,VARAS CRIMINAIS,15ª VARA CRIMINAL,Nenhuma matéria encontrada nessa data para ess...
195,DIÁRIO OFICIAL,DIÁRIO DA JUSTIÇA,JUSTIÇA ESTADUAL,COMARCA DA CAPITAL,DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VAREL...,VARAS CRIMINAIS,16ª VARA CRIMINAL,Nenhuma matéria encontrada nessa data para ess...


# O que fazer
## Mapear caminhos que aparecem na tela de conclusão
    ex: PODER EXECUTIVO -> CONTRATOS, EDITAIS E AVISOS -> SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO (SEAD)
## Seleção de data
    Colocar data na frente de cada linha
    Iterar por data (opcional)
## Converter de pandas para csv
## Tratar os links para pegar texto

# Teste

Pegar subsessões

In [103]:
elements = driver.find_element_by_id("Menu1_dgMenu_ctl02_LinkButton1")
elements.click()
tabela = driver.find_element_by_id("dgDocumentos")
subsessoes = tabela.find_elements_by_tag_name("a")
for s in subsessoes:
    print(s.text)

CONTRATOS, EDITAIS E AVISOS
GOVERNADORIA
SECRETARIAS DE ESTADO


Pegar links html

In [54]:
elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
print(elements)
elements.click()
tabela = driver.find_element_by_id("dgDocumentos")
files = tabela.find_elements_by_tag_name("a")
links = [f.get_attribute("href") for f in files]
links

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="00bc14c9-d812-4d3b-9a60-da3689727de3")>


['http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660291',
 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660292',
 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660293']

Caminhar até o fim

In [52]:
elements = driver.find_element_by_id('Menu1_dgMenu')
sessoes = elements.find_elements_by_tag_name('a')

subsessoes_1 = navegar(sessoes[0])
subsessoes_2 = navegar(subsessoes_1[0])
subsessoes_3 = navegar(subsessoes_2[0])
subsessoes_4 = navegar(subsessoes_3[0])
subsessoes_5 = navegar(subsessoes_4[0])
print(subsessoes_5)
driver.back()
driver.back()
driver.back()
#subsessoes_6 = navegar(subsessoes_1[1])

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="0bb13e6b-92a1-46b8-9527-b68ef5f5c7e1")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="621f55c5-3458-47c2-a793-12f7396cfc6d")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="32f21de6-eeba-4a32-bc08-a1a015bb5532")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="0b66c6b2-57f6-4693-a21a-8d03270e5555")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="80e6c1a8-ea54-4ed6-9e3f-3dc807c79d0c")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="68ea011a-fd01-4355-9e9a-3200c82115c8")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (sessi

In [53]:
elements = driver.find_element_by_id('Menu1_dgMenu')
elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
elements

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="927f94cd-4a14-46bd-b368-9c3c04fe8f0d")>

In [70]:
elementos = ["Menu1_dgMenu_ctl0" + str(i) + "_LinkButton1" for i in range(2,10)]
sessoes = driver.find_elements_by_id(elementos[0])
print(sessoes)
subsessoes_1 = navegar(sessoes[0])
elementos2 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" for i in range(2,len(subsessoes_1)+2)]
print(elementos2)
sessoes2 = driver.find_elements_by_id(elementos2[0])
print(sessoes2)
subsessoes_2 = navegar(sessoes2[0])
elementos3 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" for i in range(2,len(subsessoes_2)+2)]
print(elementos3)
sessoes3 = driver.find_elements_by_id(elementos3[0])
print(sessoes3)
subsessoes_3 = navegar(sessoes3[0])
subsessoes_4 = navegar(subsessoes_3[0])
subsessoes_5 = navegar(subsessoes_4[0])
print(subsessoes_5)
driver.back()
driver.back()
driver.back()
sessoes6 = driver.find_elements_by_id(elementos3[1])
subsessoes_6 = navegar(sessoes6[0])
print(subsessoes_6)
#baa570e3-c43b-47b4-9cb5-0e844586e406
#dd0912d6-54a2-42fd-a565-5212ce6bdb4d

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="54215378-e6da-4851-b111-4eb108ad7947")>]
['dgSubSessoes_ctl02_Linkbutton1', 'dgSubSessoes_ctl03_Linkbutton1', 'dgSubSessoes_ctl04_Linkbutton1']
[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="d08cdc93-a296-4326-9f1a-c8829a1e3736")>]
['dgSubSessoes_ctl02_Linkbutton1', 'dgSubSessoes_ctl03_Linkbutton1', 'dgSubSessoes_ctl04_Linkbutton1', 'dgSubSessoes_ctl05_Linkbutton1', 'dgSubSessoes_ctl06_Linkbutton1', 'dgSubSessoes_ctl07_Linkbutton1', 'dgSubSessoes_ctl08_Linkbutton1', 'dgSubSessoes_ctl09_Linkbutton1', 'dgSubSessoes_ctl010_Linkbutton1', 'dgSubSessoes_ctl011_Linkbutton1', 'dgSubSessoes_ctl012_Linkbutton1', 'dgSubSessoes_ctl013_Linkbutton1', 'dgSubSessoes_ctl014_Linkbutton1', 'dgSubSessoes_ctl015_Linkbutton1', 'dgSubSessoes_ctl016_Linkbutton1', 'dgSubSessoes_ctl017_Linkbutton1', 'dgSubSessoes_ctl018_Linkbutto